In [1]:
from tests.data_generation.stress_tests import run_evseq_stress_test

This is for stress-testing the evSeq code by passing in random inputs with known expected output. We then check to see if the evSeq outputs match the expected.

In [ ]:
run_evseq_stress_test(True, True, 
                                 keep_output = True,
                                 seed = 0)

Working on tests for seed 0...
All Uncoupled tests passed for seed: 0
All Coupled tests passed for seed: 0


Working on tests for seed 1...
Errors found for non-max Uncoupled DI03-C01 with seed 1.
Errors found for non-max Uncoupled DI05-H11 with seed 1.
Errors found for non-max Uncoupled DI02-H05 with seed 1.
Errors found for non-max Uncoupled DI04-C04 with seed 1.
Errors found for non-max Uncoupled DI04-F09 with seed 1.
Errors found for non-max Uncoupled DI02-B09 with seed 1.
Errors found for non-max Uncoupled DI04-G08 with seed 1.
Errors found for non-max Uncoupled DI02-H02 with seed 1.
Errors found for non-max Uncoupled DI02-B03 with seed 1.
Errors found for non-max Uncoupled DI04-D06 with seed 1.
Errors found for non-max Uncoupled DI03-D07 with seed 1.
Errors found for non-max Uncoupled DI01-H11 with seed 1.
Errors found for non-max Uncoupled DI06-G12 with seed 1.
Errors found for non-max Uncoupled DI02-E06 with seed 1.
Errors found for non-max Uncoupled DI03-C11 with seed 1.
Error

To do:
1. Build code to test the "max" files coming out
4. Add on code to test for DEAD wells.

In [12]:
test_run.wells[385].variants[0].mutated_positions

array([ 1,  9, 13, 32, 35, 36, 43, 47, 95])

In [15]:
test_run.wells[385].global_dead

True

In [ ]:
test_run

In [4]:
import numpy as np
np.concatenate([
    np.array([1, 2, 3]),
    np.array([1, 2, 3]) + 1
]).dtype

dtype('int64')

In [5]:
test = np.array([1,2,3])
test[-10:]

array([1, 2, 3])